# Model Testing

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
import os
os.chdir("/content/drive/MyDrive")

In [1]:
from CNN import Net
from go import GoBoard
from time import sleep
from ataxx import AtaxxBoard
from torch.optim import Adam
from model_params import MODEL_PARAMS
from az_parallel2 import AlphaZeroParallel2
from aMCTS_parallel import MCTSParallel, MCTS_Node
import numpy as np
import graphics
import pygame
import torch
torch.manual_seed(0)

import warnings
warnings.filterwarnings("ignore")


pygame 2.1.2 (SDL 2.0.18, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Load Desired Model

In [6]:
def load_model(game_type,game_size):
    model = Net(**MODEL_PARAMS.get(game_type+str(game_size)))
    
    model_path = "Final_Models/"
    model_name = game_type + str(game_size) + ".pt"
    
    model.load_state_dict(torch.load(model_path + model_name, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))
    model.eval()

    return model

### Ataxx 4x4 vs Testing

simple vs moderate -> simple

simple vs complex -> simple

moderate vs simple -> moderate

moderate vs complex -> moderate

complex vs simple -> simple

complex vs moderate -> moderate

#### complex model might be overfitting

### Model vs Minimax Testing Function

In [20]:
def face_minimax(game_type, game_size, minimax_player, mcts_iterations=100, render=False):
    board = AtaxxBoard(game_size) if game_type=="A" else GoBoard(game_size)
    board.Start(render)

    if render:
        graphics.draw_board(board.board)
        pygame.display.flip()

    model = load_model(game_type, game_size)
    mcts = MCTSParallel(model)

    while True:
        if render:
            graphics.draw_board(board.board)
            pygame.display.flip()
        print(board.board)
        #print(board.minimax(2,True,-np.inf,np.inf,board.player))

        if board.player == minimax_player:
            move = []
            if not render:
                #move = list(map(int, input("Move: ").split(" ")))
                move = board.minimax(1,True,-np.inf,np.inf,board.player)
                print("Minimax Move:", move)
                board.Move(move[0])
                board.NextPlayer()
                if render:
                    graphics.draw_board(board.board)
                    pygame.display.flip()
                board.CheckFinish(render)
            
        else:
            roots = [MCTS_Node(board)]
            mcts_probs = mcts.Search(roots, mcts_iterations, test=True)
            action = np.argmax(mcts_probs)
            move = mcts.roots[0].children[action].originMove
            print("Alphazero Move:", move)
            board.Move(move)
            board.NextPlayer()
            if render:
                graphics.draw_board(board.board)
                pygame.display.flip()
            board.CheckFinish(render)
        
        if board.hasFinished():
            print(board.board)
            if board.winner != 3:
                print(board.winner, "won")
            else:
                print("draw")
            break

In [21]:
face_minimax("A",4,2,600,True)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


MCTS Iterations:   0%|                                                    | 0/100 [00:00<?, ?iter/s]

Alphazero Move: (3, 3)
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
Minimax Move: ((0, 0), -0.5)
[[2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


MCTS Iterations:   0%|                                                    | 0/100 [00:00<?, ?iter/s]

Alphazero Move: (0, 6)
[[2. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
Minimax Move: ((0, 1), -3.5)
[[2. 2. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


...

Alphazero Move: (1, 6)
[[1. 1. 1. 2. 1. 2. 1.]
 [2. 1. 1. 2. 1. 1. 1.]
 [1. 1. 1. 1. 2. 1. 1.]
 [2. 1. 2. 1. 1. 1. 1.]
 [2. 1. 1. 1. 1. 1. 1.]
 [1. 2. 1. 1. 1. 2. 2.]
 [1. 1. 1. 1. 0. 1. 1.]]
Minimax Move: ((-1, -1), 4.5)
[[1. 1. 1. 2. 1. 2. 1.]
 [2. 1. 1. 2. 1. 1. 1.]
 [1. 1. 1. 1. 2. 1. 1.]
 [2. 1. 2. 1. 1. 1. 1.]
 [2. 1. 1. 1. 1. 1. 1.]
 [1. 2. 1. 1. 1. 2. 2.]
 [1. 1. 1. 1. 0. 1. 1.]]


MCTS Iterations:   0%|                                                    | 0/100 [00:00<?, ?iter/s]

Alphazero Move: (6, 4)
[[1. 1. 1. 2. 1. 2. 1.]
 [2. 1. 1. 2. 1. 1. 1.]
 [1. 1. 1. 1. 2. 1. 1.]
 [2. 1. 2. 1. 1. 1. 1.]
 [2. 1. 1. 1. 1. 1. 1.]
 [1. 2. 1. 1. 1. 2. 2.]
 [1. 1. 1. 1. 1. 1. 1.]]
Minimax Move: ((-1, -1), 5.5)
[[1. 1. 1. 2. 1. 2. 1.]
 [2. 1. 1. 2. 1. 1. 1.]
 [1. 1. 1. 1. 2. 1. 1.]
 [2. 1. 2. 1. 1. 1. 1.]
 [2. 1. 1. 1. 1. 1. 1.]
 [1. 2. 1. 1. 1. 2. 2.]
 [1. 1. 1. 1. 1. 1. 1.]]


MCTS Iterations:   0%|                                                    | 0/100 [00:00<?, ?iter/s]

Alphazero Move: (-1, -1)
[[1. 1. 1. 2. 1. 2. 1.]
 [2. 1. 1. 2. 1. 1. 1.]
 [1. 1. 1. 1. 2. 1. 1.]
 [2. 1. 2. 1. 1. 1. 1.]
 [2. 1. 1. 1. 1. 1. 1.]
 [1. 2. 1. 1. 1. 2. 2.]
 [1. 1. 1. 1. 1. 1. 1.]]
2 won
